In [1]:
import sqlite3
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker()

def generate_valid_name():
    while True:
        vendor_name = fake.name()
        name_parts = vendor_name.split()

        # Ensure the generated name contains exactly 2 elements
        if len(name_parts) == 2:
            return vendor_name

def generate_correlated_email(vendor_name, domain):
    
    # Split the vendor name into first name and last name
    first_name, last_name = vendor_name.split()

    # Take the first character of the first name and the entire last name
    email_prefix = f"{first_name[0]}_{last_name}"

    # Make the email lowercase
    email_prefix = email_prefix.lower()

    return f"{email_prefix}@{domain}"

def generate_phone_number():
    random_nums = fake.random_number(digits=8)  # Generate a random 7-digit number

    return f"6{random_nums}"

def generate_salary(v_expertise):
    if v_expertise == 'Junior':
        v_salary = random.randint(20000, 30000);
    elif v_expertise == 'Senior':
        v_salary = random.randint(30000, 60000);

    return v_salary

country_cities = {
    'Spain': ['Barcelona', 'Barcelona', 'Barcelona', 'Malaga', 'Malaga', 'Madrid', 'Valencia', 'Bilbao', 'Lugo', 'Badajoz'],
    'France': ['Paris'],
    'Italy': ['Rome', 'Turin', 'Venice'],
}

# Function to generate correlated city based on country
def generate_correlated_city(country):
    return random.choice(country_cities[country])

def generate_order_details_list():

    order_details_list = []
    selected_product_ids = set()
    
    # Generate a random number of products for the order (e.g., between 1 and 5)
    num_products = random.randint(1, 5)

    for _ in range(num_products):
        # Choose a random product ID from the list
        product_id = random.choice([pid for pid in range(1, (NUM_PRODUCTS+1)) if pid not in selected_product_ids])
        selected_product_ids.add(product_id)

        # Generate a random quantity for the product (e.g., between 1 and 10)
        quantity = random.randint(1, 10)

        # Add the product details to the order_details_list
        order_details_list.append({'ProductID': product_id, 'Quantity': quantity, 'Price': 0})

    return order_details_list

# Connect to SQLite database
conn = sqlite3.connect('InnovaTech.db')
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE Vendors (
        VendorID INTEGER PRIMARY KEY AUTOINCREMENT,
        VendorName VARCHAR(100),
        Expertise VARCHAR(50),
        Salary DECIMAL(10, 2),
        Email VARCHAR(100),
        Phone VARCHAR(20)
    )
""")

cursor.execute("""
    CREATE TABLE Customers (
        CustomerID INTEGER PRIMARY KEY AUTOINCREMENT,
        StoreName VARCHAR(100),
        City VARCHAR(50),
        Country VARCHAR(50),
        ContactPerson VARCHAR(100),
        Email VARCHAR(100),
        Phone VARCHAR(20)
    )
""")

cursor.execute("""
    CREATE TABLE Products (
        ProductID INTEGER PRIMARY KEY AUTOINCREMENT,
        Category VARCHAR(50),
        Brand VARCHAR(50),
        RAM VARCHAR(10),
        StorageCapacity VARCHAR(20),
        Processor VARCHAR(50),
        Price DECIMAL(10, 2),
        QuantityInStock INT
    )
""")

cursor.execute("""
    CREATE TABLE Orders (
        OrderID INTEGER PRIMARY KEY AUTOINCREMENT,
        CustomerID INT,
        VendorID INT,
        OrderDate DATE,
        TotalOrderPrice DECIMAL(10, 2),
        FinalStatus VARCHAR(20),
        FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID),
        FOREIGN KEY (VendorID) REFERENCES Vendors(VendorID)
    )
""")

cursor.execute("""
    CREATE TABLE OrderItems (
        OrderItemID INTEGER PRIMARY KEY AUTOINCREMENT,
        OrderID INT,
        ProductID INT,
        Quantity INT,
        FOREIGN KEY (OrderID) REFERENCES Orders(OrderID),
        FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
    )
""")

cursor.execute("CREATE INDEX idx_OrderDate ON Orders(OrderDate)")

# Define the size of the database and other varibles

NUM_VENDORS = 7
NUM_CUSTOMERS = 22
NUM_PRODUCTS = 40
NUM_ORDERS = 400

for _ in range(NUM_VENDORS):
    v_name = generate_valid_name()
    v_expertise = fake.random_element(elements=('Junior', 'Senior'))
    cursor.execute("""
        INSERT INTO Vendors (VendorName, Expertise, Salary, Email, Phone)
        VALUES (?, ?, ?, ?, ?)
    """, (
        v_name,
        v_expertise,
        generate_salary(v_expertise),
        generate_correlated_email(v_name, 'innovatech.com'),
        generate_phone_number(),
    ))
    
for _ in range(NUM_CUSTOMERS):
    c_country = random.choices(['Spain', 'France', 'Italy'], weights=[7, 1, 2])[0]
    c_name = generate_valid_name()
    cursor.execute("""
        INSERT INTO Customers (StoreName, City, Country, ContactPerson, Email, Phone)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (
        fake.company(),
        generate_correlated_city(c_country),
        c_country,
        c_name,
        generate_correlated_email(c_name, fake.random_element(elements=('gmail.com', 'yahoo.com'))),
        generate_phone_number()
    ))
    
categories = ['Desktop', 'Laptop']
brands = ['Apple', 'Dell', 'Acer', 'ASUS', 'HP']
rams = ['8GB', '16GB', '32GB', '64GB']
storage_capacities = ['512GB SSD', '512GB HDD', '1TB HDD']
processors = ['Intel Core i7', 'Intel Core i9', 'AMD Ryzen 7', 'AMD Ryzen 9']

unique_combinations = set()

while len(unique_combinations) < NUM_PRODUCTS:
    category = fake.random_element(elements=categories)
    brand = fake.random_element(elements=brands)
    ram = fake.random_element(elements=rams)
    storage_capacity = fake.random_element(elements=storage_capacities)
    processor = fake.random_element(elements=processors)

    combination = (category, brand, ram, storage_capacity, processor)
    unique_combinations.add(combination)

# Generate products with adjusted prices based on specifications
for combination in unique_combinations:
    category, brand, ram, storage_capacity, processor = combination

    price_base = 100
    quantity_in_stock = random.randint(0, 100)

    # Adjust price based on specifications
    price_adjustment = {
        '8GB': 50,
        '16GB': 150,
        '32GB': 250,
        '64GB': 400,
        '512GB SSD': 120,
        '512GB HDD': 50,
        '1TB HDD': 150,
        'Intel Core i7': 70,
        'Intel Core i9': 400,
        'AMD Ryzen 7': 50,
        'AMD Ryzen 9': 350,
        'Apple': 170,
        'Dell': 120,
        'Acer': 50,
        'ASUS': 70,
        'HP': 60,
        'Desktop': 50,
        'Laptop': 300,
    }

    price = price_base + sum(price_adjustment.get(spec, 0) for spec in (ram, storage_capacity, processor, category, brand))

    cursor.execute("""
        INSERT INTO Products (Category, Brand, RAM, StorageCapacity, Processor, Price, QuantityInStock)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (
        category,
        brand,
        ram,
        storage_capacity,
        processor,
        price,
        quantity_in_stock,
    ))
    
for _ in range(NUM_ORDERS):
    # Step 1: Create the Order
    
    customer_id = random.randint(1, NUM_CUSTOMERS)
    vendor_id = random.randint(1, NUM_VENDORS)

    cursor.execute("""
        INSERT INTO Orders (CustomerID, VendorID, OrderDate, TotalOrderPrice, FinalStatus)
        VALUES (?, ?, ?, 0, ?)
    """, (
        customer_id,
        vendor_id,
        fake.date_between_dates(date_start=datetime(2021, 1, 1), date_end=datetime(2024, 1, 1)),
        random.choices(['Delivered', 'Canceled', 'Lost'], weights=[95, 4, 1])[0],
    ))

    # Retrieve the OrderID of the inserted order
    order_id = cursor.lastrowid

    # Step 2: Create the OrderItems
    order_details_list = generate_order_details_list()

    # Insert order items into the OrderItems table
    for product in order_details_list:
        # Insert the order item into the database
        cursor.execute("""
            INSERT INTO OrderItems (OrderID, ProductID, Quantity)
            VALUES (?, ?, ?)
        """, (
            order_id, 
            product['ProductID'], 
            product['Quantity']))    
        
    # Fetch the actual price from the Products table
        cursor.execute("""
            SELECT Price
            FROM Products
            WHERE ProductID = ?
        """, (product['ProductID'],))
        product_price = cursor.fetchone()[0]

        # Update the 'Price' information in the order_details_list
        product['Price'] = product_price

    # Calculate dynamic price
    dynamic_price = sum(
        product['Price'] * product['Quantity']
        for product in order_details_list
    )
        
    # Update the TotalOrderPrice in the Orders table
    cursor.execute("""
        UPDATE Orders
        SET TotalOrderPrice = ?
        WHERE OrderID = ?
    """, (dynamic_price, order_id))
    
conn.commit()
conn.close()